In [0]:
%pylab inline

In [0]:
import os
project_key = os.getenv("DKU_CURRENT_PROJECT_KEY")

In [0]:
import dataiku
import dataikuapi
client = dataiku.api_client()
project = client.get_default_project()
DATASET_NAME = "iris"
input_key = "index"
original_ds = dataiku.Dataset(DATASET_NAME)
original_df = original_ds.get_dataframe()
connection_name = original_ds.get_config()['params']['connection'] # nom de la connexion SQL dataiku où aller récupérer la table
changes_ds_name = DATASET_NAME + "_changes"
editable_ds_name = DATASET_NAME + "_editable"

In [0]:
client.get_own_user().get_settings().get_raw()

## Create new datasets

In [0]:
changes_ds_creator = dataikuapi.dss.dataset.DSSManagedDatasetCreationHelper(project, changes_ds_name)
changes_ds_creator.with_store_into(connection="filesystem_managed")
changes_ds_creator.create()

In [0]:
changes_ds = dataiku.Dataset(changes_ds_name)

In [0]:
changes_ds.write_schema_from_dataframe(df=original_df)

In [0]:
editable_ds_creator = dataikuapi.dss.dataset.DSSManagedDatasetCreationHelper(project, editable_ds_name)
editable_ds_creator.with_store_into(connection=connection_name) # TODO: make this configurable
editable_ds_creator.create()

In [0]:
editable_ds = dataiku.Dataset(editable_ds_name)

In [0]:
editable_ds.write_with_schema(original_df)

## Get table name

In [0]:
editable_tablename = editable_ds.get_config()['params']['table'].replace("${projectKey}", project_key)
print(editable_tablename)

## SQL queries

### Update row

In [0]:
idx = 0
col_id = "sepal_length"
val = 0

In [0]:
query = """UPDATE \"{0}\" SET {1}={2}
            WHERE {3}={4}
            RETURNING {1};
            COMMIT;
            """.format(editable_tablename, col_id, val, input_key, idx)
print(query)

In [0]:
from dataiku.core.sql import SQLExecutor2
executor = SQLExecutor2(connection=connection_name)

In [0]:
result_df = executor.query_to_df(query)
result_df

### Retrieve lookup columns

In [0]:
ext_lookup_columns = ["sepal_length", "sepal_width"]
ext_tablename = editable_tablename
ext_key = "index"
ext_idx = "0"

In [0]:
select_query = "SELECT " \
    + ", ".join(ext_lookup_columns) \
    + " FROM \"" + ext_tablename \
    + "\" WHERE " + ext_key + "=" + ext_idx
select_df = executor.query_to_df(select_query)

In [0]:
val = select_df.iloc[0][ext_lookup_columns[0]]
print(val)

### Update lookup columns

In [0]:
update_query = "UPDATE \"" + editable_tablename \
    + "\" SET " + ", ".join([col + "=" + str(select_df.iloc[0][col]) for col in ext_lookup_columns]) \
    + " WHERE " + ext_key + "=" + ext_idx \
    + "; COMMIT;"
print(update_query)

In [0]:
executor.query_to_df(update_query)

## Create a recipe

In [0]:
recipe_creator = dataikuapi.dss.recipe.DSSRecipeCreator("CustomCode_sync-and-apply-changes", "compute_iris_editable", project)
recipe = recipe_creator.create()

In [0]:
settings = recipe.get_settings()
settings.add_input("input", "iris")
settings.add_input("changes", "iris_changes")
settings.add_output("editable", "iris_editable")
settings.raw_params["customConfig"] = {"key": "index"}
settings.save()

Alternative:

```python
recipe_creator.with_input("iris", role="input")
recipe_creator.with_input("iris_changes", role="changes")
recipe_creator.with_output("iris_editable", role="editable")
```

In [0]:
recipe = project.get_recipe("compute_iris_editable")

In [0]:
settings = recipe.get_settings()